In [12]:
import pickle
import pandas as pd
import numpy as np

In [3]:
df = pickle.load(open('/home/rc7ne/Project/SessionRecommendation/Data/tmall/100000_cate/10w_cate.pickle', 'rb'))

In [6]:
df.head()

,userid,itemid,categoryid,behavior,timestamp
41582965,16,3840502,982926,pv,1511626494
41582966,16,3716887,3607361,pv,1511626551
41582968,16,3840502,982926,pv,1511626597
41582969,16,4115794,982926,pv,1511626761
41582970,16,2111454,570735,pv,1511626770


In [8]:
itemids = df.itemid.unique()

In [10]:
categoryids = df.categoryid.unique()

In [18]:
category_map = {}
idx = 0
for c in categoryids:
    category_map[c] = idx
    idx += 1

2739 2739


In [37]:
adj = np.zeros((len(categoryids), len(categoryids)), dtype=int)

In [38]:
print(adj[1][1])

0


In [39]:
for i in itemids:
    same_category = df[df.itemid == i].categoryid.unique()
    N = len(same_category)
    if N > 1:
#         print('found', same_category)
        for i in range(N):
            c1 = category_map[ same_category[i] ]
            for j in range(i, N):
                c2 = category_map[ same_category[j] ]
                adj[c1][c2] = 1
                adj[c2][c1] = 1

found [2920476 2355072]
found [4818107 2834771]
found [3090850 3277224]
found [1080785 4756105]
found [1884322 3158249]
found [4756105 1080785]
found [1080785 4756105]
found [2367907 4202143]
found [1884322 3158249]
found [3158249 1884322]
found [2520377 1320293]
found [3002561 4690421]
found [2476560 4138583]
found [ 983613 2737340]
found [ 323851 2558244]
found [4145813 3102419]
found [ 154040 2355072]
found [2945933 3454970]
found [4145813 3102419]
found [5063620 3154173]
found [4818107  200278]
found [4818107  200278]
found [401357 383530]
found [ 200278 3091480]
found [3818081  982926]
found [1834026 3158249]
found [4138583 2476560]
found [3461823 4294583]
found [4794845  902410]
found [ 665090 3694648]
found [3248864 3880463]
found [4145813 3002561]
found [4308141 2296439]
found [1376358 4060157]
found [ 901282 4718907]
found [2476560 4138583]
found [2920476 1320293]
found [4145813 3102419]
found [3409740 1169503]
found [ 903809 3065645]
found [4801426 2735466]
found [ 751494 369

In [41]:
print(adj.nonzero())

(array([   0,    0,    3,    3,    3,   10,   10,   11,   11,   11,   11,
         19,   19,   19,   20,   20,   26,   26,   28,   28,   28,   49,
         49,   49,   60,   60,   60,   61,   61,   64,   64,   64,   65,
         65,   72,   72,   83,   83,   83,   86,   86,   86,   93,   93,
        104,  104,  105,  105,  128,  128,  151,  151,  154,  154,  184,
        184,  195,  195,  206,  206,  207,  207,  228,  228,  228,  235,
        235,  238,  238,  272,  272,  285,  285,  285,  286,  286,  330,
        330,  331,  331,  333,  333,  343,  343,  359,  359,  374,  374,
        381,  381,  427,  427,  454,  454,  468,  468,  516,  516,  542,
        542,  547,  547,  588,  588,  600,  600,  614,  614,  623,  623,
        642,  642,  746,  746,  755,  755,  790,  790,  836,  836,  863,
        863,  948,  948, 1013, 1013, 1079, 1079, 1218, 1218, 1274, 1274,
       1307, 1307, 1333, 1333, 1537, 1537, 1550, 1550, 1648, 1648, 1852,
       1852, 1995, 1995, 2437, 2437, 2528, 2528, 2

In [52]:
groupids = np.zeros((len(categoryids)), int)
gid = 1
def dfs(idx, gid):
    if groupids[idx] != 0: 
        return
#     print(adj[idx].nonzero())
    for n in adj[idx].nonzero()[0]:
        groupids[idx] = gid
        dfs(n, gid)
        
for i in range(len(categoryids)):
    if groupids[i] == 0:
        dfs(i, gid)
        groupids[i] = gid
        gid += 1


In [53]:
print( len(groupids), len(set( groupids)) )

2739 2698


In [55]:
df['categoryid'] = df['categoryid'].apply(lambda x: groupids[ category_map[x] ])

In [56]:
df.head()

,userid,itemid,categoryid,behavior,timestamp
41582965,16,3840502,1,pv,1511626494
41582966,16,3716887,2,pv,1511626551
41582968,16,3840502,1,pv,1511626597
41582969,16,4115794,1,pv,1511626761
41582970,16,2111454,3,pv,1511626770


In [57]:
df.categoryid.nunique()

2698

In [58]:
pickle.dump(df, open('10w_cate_nodup.pickle', 'wb'))